# Dummy Variablen

In [ ]:
if(!require("dplyr")) install.packages("dplyr")
if(!require("ggplot2")) install.packages("ggplot2")
if(!require("caret")) install.packages("caret")
if(!require("miscTools")) install.packages("miscTools")

Beispiel für die Verwendung von Dummy-Variablen! Wir laden einen Beispiel-Datensatz und betrachten die Struktur der Daten:

In [ ]:
daten <- read.csv("daten_dummy.csv", sep=";")
daten
str(daten)

Der Datensatz enthält Daten über Mietpreise von Wohnungen in Abhängigkeit von der Größe in $m^2$ und der Lage. Die Lage ist dabei ein klassisches Beispiel für nominal skalierte Daten. Dieses Feature ist auch schon entsprechend codiert worden: Die Ausgabe der *str*-Funktion zeigt beim Feature *Lage* den Datentyp "Factor w(ith) 3 Levels, konkret sind das *Innenstadt*, *Außenbezirk* und *Umland*.



Zuerst einige grafische Auswertungen: Wir geben in einem Plot die Mietpreise in Abhängigkeit der Wohnungsgröße und der Lage aus:

In [ ]:
daten %>% ggplot(aes(Quadratmeter, Mietpreis, color=Lage)) +
  geom_point(size=3) +
  geom_smooth(method="lm", se=F)

Zur Übersicht geben wir diese Daten auch noch als Boxplot aus:

In [ ]:
daten %>% ggplot(aes(Quadratmeter, Mietpreis, fill=Lage)) + geom_boxplot()

Wir erstellen nun ein Modell und sagen den Mietpreis einer Wohnung voraus:

In [ ]:
model <- lm(Mietpreis~., data=daten)
model
summary(model)

Wir können nun anhand unseres Modells für Wohnungen Mietpreise vorhersagen. Angenommen ein Makler hat 3 Wohnungen im Angebot und möchte sich die Mietpreise schätzen lassen. Man könnte nun wie folgt vorgehen:

In [ ]:
# Dateframe mit 3 (neuen) Wohnungen erstellen:
neue.wohnungen = data.frame(Quadratmeter=c(40, 79, 50),
                           Lage = factor(c("Innenstadt", "Innenstadt", "Umland")))
print(neue.wohnungen)

# Mietpreise schätzen:
round(predict(model, neue.wohnungen))

## Bewertung unseres Modells

Wir wollen nun nochmals das Modell erstellen, jetzt allerdings anhang von Trainings- und Testdaten:

In [ ]:
# Trainings- und Testdaten
index <- createDataPartition(daten$Mietpreis, p=0.8, list=FALSE)
train <- daten[index, ]
test  <- daten[-index, ]

# Modell mit Trainingsdaten erstellen
model2 <- lm(Mietpreis~., data=train)

# Prognose erstellen
pred <- predict(model2, test)

# Nun berechnen wir den Korrelationskoefizienten anhand der Daten im Testdatensatz und der Prognose.
# Hierfür verwenden wir die Funktion rSquared aus dem Package miscTools. Diese Funktion erwartet als
# erstes Argument die y-Werte und als 2. Argument die Residuen:

rSquared(test$Mietpreis, pred-test$Mietpreis)